<a href="https://colab.research.google.com/github/TheDenk/Attention-Interpolation/blob/main/jupyters/colab_example_attention_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/TheDenk/Attention-Interpolation.git
!pip install -r ./Attention-Interpolation/requirements.txt > /dev/null

fatal: destination path 'Attention-Interpolation' already exists and is not an empty directory.


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append('./Attention-Interpolation')

import cv2
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from omegaconf import OmegaConf
from diffusers import ControlNetModel, DDIMScheduler, StableDiffusionPipeline

from iattention import UNION_PIPELINES
from iattention.utils import correct_colors_hist, show_image, show_images

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


In [ ]:
def get_video_info(cap):
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return height, width, fps, frame_count

In [ ]:
config = OmegaConf.load('./Attention-Interpolation/configs/default.yaml')
config['pipe_config']['total_steps'] = config['common']['num_inference_steps']
config['unet_config']['total_steps'] = config['common']['num_inference_steps']
config['controlnet_config']['total_steps'] = config['common']['num_inference_steps']
config['common']['unet_from'] = None #'./Attention-Interpolation/models/deliberate_v2.safetensors'

In [ ]:
pipe = UNION_PIPELINES[config['pipeline']](config)

In [ ]:
config['common']['input_video_path'] = './Attention-Interpolation/video/input/man.mp4'
config['common']['output_video_path'] = './Attention-Interpolation/video/output/video-from-jupyter.mp4'

cap = cv2.VideoCapture(config['common']['input_video_path'])
orig_height, orig_width, fps, frame_count = get_video_info(cap)

print(f'\n\n[ VIDEO INFO | WxH: {orig_width}x{orig_height} | FPS: {fps} | FRAME COUNT: {frame_count} ]\n\n')

img_h, img_w = config['common']['img_h'], config['common']['img_w']
out_h, out_w = img_h, img_w
if config['common']['original_output_size']:
    out_h, out_w = orig_height, orig_width

writer = cv2.VideoWriter(
    config['common']['output_video_path'],
    cv2.VideoWriter_fourcc(*'mp4v'),
    fps,
    (out_w * 2, out_h),
)

first_image = None
for _ in tqdm(range(frame_count)):
    c_ret, c_image = cap.read()
    if not c_ret or c_image is None:
        break
    image = cv2.resize(c_image, (img_h, img_w))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    result = pipe(image)

    if first_image is None:
        first_image = result.copy()
    else:
        predict_image = correct_colors_hist(first_image, result, config['common']['hist_normalize'])

    out_images = [cv2.resize(x, (out_w, out_h)) for x in [image, result]]
    result = np.hstack(out_images)
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)

    writer.write(result)


cap.release()
writer.release()



[ VIDEO INFO | WxH: 750x720 | FPS: 30 | FRAME COUNT: 36 ]




100%|██████████| 36/36 [09:14<00:00, 15.41s/it]
